In [1]:
import torch
import numpy as np
import pandas as pd
import helper_functions as hf
from dataloader import JPXData
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataloader import default_collate

device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device}")

Using cuda


In [2]:
# Run this cell to create the new modified_data folder
# ~ 1 min runtime

# hf.concat_data()


In [3]:
# Run this cell to tokenize Section/Product and NewMarketSegmentColumns on stock_list.csv
# ~ 20 secs runtime

# hf.tokenize_stock_list()

In [4]:
batch_size = 1
data = JPXData(batch_size=batch_size)
trainloader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=False, num_workers=2)

In [5]:
data[0]

(tensor([2734.,  568., 3150.,  ...,    0.,    0.,    0.]),
 tensor([2743.,  572., 3225.,  ...,    0.,    0.,    0.]))

In [6]:
# for i in range(1000):
#     a = data[i]
#     x = list(a[0]["SecuritiesCode"])
#     y = list(a[1]["SecuritiesCode"])


In [7]:
stock_list = pd.read_csv("modified_data" + '/' + "tokenized_stock_list.csv")

In [8]:
print(len(stock_list))

4417


In [9]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Linear(2000, 500),
            nn.ReLU(),
            nn.Linear(500, 500),
            nn.ReLU(),
            nn.Linear(500, 2000),

        )

    def forward(self, x):
        output = self.layer_stack(x)
        return output

In [10]:
net = Net().to(device)
print(net)

Net(
  (layer_stack): Sequential(
    (0): Linear(in_features=2000, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=500, out_features=2000, bias=True)
  )
)


In [11]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)

In [12]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_addmm)